In [1]:
import numpy as np
from matplotlib import pyplot as plt
import meshio
import tetgen
from Mesh_tetra import Mesh
import pyvista as pv
from scipy.linalg import eigh
from jax import vmap
import jax.numpy as jnp
from jax import random
from mpl_toolkits.mplot3d import Axes3D

The hexaedral mesh nodes and elements are imported.

In [2]:
nodes = np.genfromtxt('DENSE/S7/S7.node', skip_header=True)
connectivity = np.genfromtxt('DENSE/S7/S7.ele', skip_header=True).astype(int)

Fibers are imported.

In [3]:
invivo_fiber = np.genfromtxt('DENSE/S7/S7_invivo.Fiber')

Now, the quadrature points are computed for an hexaedral element. According to the author of the dataset, this quadrature points follows the abaqus C3D8 norm.

In [4]:
def nodes_connectivity(points, connectivity):
    return points[connectivity]

nodes_connectivity_fn = lambda x: nodes_connectivity(jnp.array(nodes), x)
hexaedrons_nodes = np.squeeze(np.array(vmap(nodes_connectivity_fn)(connectivity)))

In [5]:
isoparametric_nodes = jnp.array([[1, -1 , -1],
                                [1, 1, -1],
                                [-1, 1, -1],
                                [-1, -1, -1],
                                [1, -1, 1],
                                [1, 1, 1],
                                [-1, 1, 1],
                                [-1, -1, 1]])

quadrature_points = isoparametric_nodes*(1/jnp.sqrt(3))

def phi_i(isoparametric_node, quadrature_point):
    xi, eta, zeta = isoparametric_node
    xi_i, eta_i, zeta_i = quadrature_point
    return (1/8)*(1 + xi*xi_i)*(1 + eta*eta_i)*(1 + zeta*zeta_i)


Phi_matrix = []
for quadrature_point in quadrature_points:
    phi_i_fn = lambda x: phi_i(x, quadrature_point)
    phis = np.array(vmap(phi_i_fn)(isoparametric_nodes))
    Phi_matrix.append(phis)

Phi_matrix = np.array(Phi_matrix)



def int_points(phi, hexaedron):
    return jnp.dot(phi, hexaedron)


interpolation_points = []
for hexaedron in hexaedrons_nodes:
    int_node_fn = lambda x: int_points(x, hexaedron)
    int_node = np.squeeze(np.array(vmap(int_node_fn)(Phi_matrix)))
    interpolation_points.append(int_node)

interpolation_points = np.array(interpolation_points)
interpolation_points = interpolation_points.reshape((-1, 3))

In [6]:
np.savetxt('Quadrature/hexaedrons_points.csv', hexaedrons_nodes.reshape((-1, 3)))
np.savetxt('Quadrature/quadrature_points.csv', interpolation_points)

In [7]:
interpolation_points = interpolation_points.reshape((-1, 3))

It is checked that the number of quadrature points is the same as the number of points where the fibers are known.

In [8]:
print("Number of quadrature points:", interpolation_points.shape[0], "\nNumber of fiber points:", invivo_fiber.shape[0])

Number of quadrature points: 9504 
Number of fiber points: 9504


Fibers are exported to be plot in Paraview.

In [9]:
first_eigs_invivo = invivo_fiber[:, :3]
np.savetxt('Quadrature/first_fibers_invivo.csv', np.concatenate((interpolation_points, first_eigs_invivo), axis = 1))

To visualize them in Paraview:
* Import the csv, uncheck the box of  $Have \; Headers$  and change $Field \; Delimiter \; Character$ for a space (" ").
* Apply $Table \; to \; Points$ field, selecting field 0, field 1 y field 2.
* Apply a $Calculator$ as follows:
$$"Field \; 3"*iHat + "Field \; 4"*jHat + "Field \; 5"*kHat$$
"Field" parameter is found in the $Scalars$ display menu, located under the calculator operations.
* Apply a $Glyph$ filter to this $Calculator$.